# Init

In [17]:
!pip install gensim

In [18]:
# geral
import pickle
import pandas as pd

# gensim
import gensim.corpora as corpora

In [19]:
with open('pickle/df_proc.p', 'rb') as file:
    df = pickle.load(file)
df.head()

,text
0,programa treinamento jornalismo agroindústria ...
1,ofício enviado sergio presidente michel esquiv...
2,governador sérgio cabral transferido complexo ...
3,aplausos federal sergio responsável operação i...
4,executivo empreiteira odebrecht afirmou acordo...


In [20]:
with open('pickle/optimal_lda.p', 'rb') as file:
    optimal_lda = pickle.load(file)

# Anexar classificação de tópicos no dataset

In [21]:
tokenized_docs = [doc.lower().split() for doc in df['text']]
id2word = corpora.Dictionary(tokenized_docs)
corpus = [id2word.doc2bow(text) for text in tokenized_docs]

In [22]:
def get_document_topic(lda_model, corpus):
    document_topics = []
    for bow in corpus:
        topic_probs = lda_model.get_document_topics(bow, minimum_probability=0.0)
        dominant_topic = max(topic_probs, key=lambda x: x[1])[0]
        document_topics.append(dominant_topic)
    return document_topics

In [23]:
target = get_document_topic(optimal_lda, corpus)

In [24]:
df['target'] = target

In [27]:
df

,text,target
0,programa treinamento jornalismo agroindústria ...,10
1,ofício enviado sergio presidente michel esquiv...,6
2,governador sérgio cabral transferido complexo ...,6
3,aplausos federal sergio responsável operação i...,6
4,executivo empreiteira odebrecht afirmou acordo...,6
...,...,...
10092,produtor executivo retrata indústria fonográfi...,4
10093,novato começa direito sauber equipe honesta es...,12
10094,eletrizam promessa reviravoltas capricórnio nu...,10
10095,brisbane bandsports bandsports primavera flumi...,4


In [26]:
import pickle
with open('pickle/df_target.p', 'wb') as file:
    pickle.dump(df, file)